In [1]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
station = pd.read_csv('./data/seoul_bus_station.csv', encoding='cp949')
boarding = pd.read_csv('./data/2021_boarding_timely.csv', encoding='cp949')

C:\Users\cjkang\AppData\Local\Temp\ipykernel_18680\1783597532.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  boarding = pd.read_csv('./data/2021_boarding_timely.csv', encoding='cp949')


In [9]:
station.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70835 entries, 0 to 70834
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   정류장_ID           70835 non-null  int64  
 1   정류장_명칭           70835 non-null  object 
 2   정류장_유형           70599 non-null  object 
 3   정류장_번호           70815 non-null  float64
 4   위도               70832 non-null  float64
 5   경도               70832 non-null  float64
 6   버스도착정보안내기_설치_여부  70835 non-null  object 
dtypes: float64(3), int64(1), object(3)
memory usage: 3.8+ MB


In [4]:
station.head()

,정류장_ID,정류장_명칭,정류장_유형,정류장_번호,위도,경도,버스도착정보안내기_설치_여부
0,717104358,계수사거리(경유),가상정류장,0.0,126.817473,37.452473,미설치
1,500000288,행사장입구,마을버스,25965.0,126.691556,37.321742,미설치
2,500000287,행사장입구,마을버스,25095.0,126.691728,37.321942,미설치
3,500000286,아남중전기,일반차로,25963.0,126.710485,37.319934,미설치
4,500000285,3Z,일반차로,25964.0,126.707130,37.320566,미설치


In [10]:
drop_cols = ['정류장_명칭', '정류장_유형', '정류장_번호']
station = station.drop(drop_cols, axis=1)

station.head()

,정류장_ID,위도,경도,버스도착정보안내기_설치_여부
0,717104358,126.817473,37.452473,미설치
1,500000288,126.691556,37.321742,미설치
2,500000287,126.691728,37.321942,미설치
3,500000286,126.710485,37.319934,미설치
4,500000285,126.707130,37.320566,미설치


In [14]:
station = station.dropna()
station = station.reset_index(drop=True)
station.tail()

,정류장_ID,위도,경도,버스도착정보안내기_설치_여부
70827,100000005,127.001744,37.586243,설치
70828,100000004,126.987613,37.568579,설치
70829,100000003,126.998340,37.582671,설치
70830,100000002,126.996566,37.579183,설치
70831,100000001,126.987750,37.569765,설치


# Kakao REST API

In [29]:
# 1. API 문서 : URL
API_KEY = '0ca60ee4f7ac2204b680fa9f7777708e'
latitude = station.loc[0, '위도']
longitude = station.loc[0, '경도']
url = f'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json'
headers = {
    'Authorization' : f'KakaoAK {API_KEY}'
}
params = {
    'x' : latitude,
    'y' : longitude,
    'input_coord' : 'WGS84'
}

In [30]:
# request(url, key, coord) -> response : json(str)
response = requests.get(url, params=params, headers=headers)
response

<Response [200]>

In [35]:
# json(str) -> dict -> hgd
json = response.json()
json['documents'][1]

{'region_type': 'H',
 'code': '4139051000',
 'address_name': '경기도 시흥시 대야동',
 'region_1depth_name': '경기도',
 'region_2depth_name': '시흥시',
 'region_3depth_name': '대야동',
 'region_4depth_name': '',
 'x': 126.78853828587953,
 'y': 37.44304606566207}

In [58]:
# make function

def get_hgd(x, y):
    API_KEY = '0ca60ee4f7ac2204b680fa9f7777708e'
    URL = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json'
    
    headers = {
    'Authorization' : f'KakaoAK {API_KEY}'
}
    params = {
        'x' : x,
        'y' : y,
        'input_coord' : 'WGS84'
    }
    
    response = requests.get(URL, params=params, headers=headers)
    json = response.json()
    try: 
        result = (json['documents'][1]['region_2depth_name'], json['documents'][1]['region_3depth_name'])
    except:
        result = ('', '')
    return result

In [43]:
station_hgd = station.copy()

In [47]:
station_hgd['구이름'] = ''
station_hgd['행정동'] = ''
station_hgd.head()

,정류장_ID,위도,경도,버스도착정보안내기_설치_여부,구이름,행정동
0,717104358,126.817473,37.452473,미설치,,
1,500000288,126.691556,37.321742,미설치,,
2,500000287,126.691728,37.321942,미설치,,
3,500000286,126.710485,37.319934,미설치,,
4,500000285,126.707130,37.320566,미설치,,


In [59]:
for i in range(len(station_hgd)):
    if station_hgd.loc[i, '구이름'] != "":
        continue
    latitude = station_hgd.loc[i, '위도']
    longitude = station_hgd.loc[i, '경도']
    
    result = get_hgd(latitude, longitude)
    
    station_hgd.loc[i, '구이름'], station_hgd.loc[i, '행정동'] = result

station_hgd.tail()

,정류장_ID,위도,경도,버스도착정보안내기_설치_여부,구이름,행정동
70827,100000005,127.001744,37.586243,설치,종로구,혜화동
70828,100000004,126.987613,37.568579,설치,종로구,종로1.2.3.4가동
70829,100000003,126.998340,37.582671,설치,종로구,혜화동
70830,100000002,126.996566,37.579183,설치,종로구,종로1.2.3.4가동
70831,100000001,126.987750,37.569765,설치,종로구,종로1.2.3.4가동


In [60]:
station_hgd.to_csv('./data/seoul_bus_station_hgd.csv', sep=',')

In [65]:
check = pd.read_csv('./data/seoul_bus_station_hgd.csv', index_col=0)
check.head()

,정류장_ID,위도,경도,버스도착정보안내기_설치_여부,구이름,행정동
0,717104358,126.817473,37.452473,미설치,시흥시,대야동
1,500000288,126.691556,37.321742,미설치,시흥시,정왕2동
2,500000287,126.691728,37.321942,미설치,시흥시,정왕2동
3,500000286,126.710485,37.319934,미설치,시흥시,정왕2동
4,500000285,126.707130,37.320566,미설치,시흥시,정왕2동


# 광진구만 남기기

In [66]:
station_gwangjin = station_hgd.loc[station_hgd['구이름'] == '광진구']
station_gwangjin.head()

,정류장_ID,위도,경도,버스도착정보안내기_설치_여부,구이름,행정동
1752,277103552,127.067091,37.531727,미설치,광진구,자양3동
1753,277103551,127.066757,37.531816,미설치,광진구,자양4동
1793,277103511,127.059683,37.533713,미설치,광진구,자양4동
1794,277103510,127.088280,37.528064,미설치,광진구,자양2동
1795,277103509,127.088549,37.528812,미설치,광진구,자양2동


In [67]:
station_gwangjin.to_csv('./data/gwangjin_bus_station.csv', index=False)

In [74]:
station_gwangjin = station_gwangjin.rename(columns={"정류장_ID":"표준버스정류장ID"})

# 시간대별 승하차 정보

In [7]:
boarding.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 469361 entries, 0 to 469360
Data columns (total 55 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   사용년월        469361 non-null  int64 
 1   노선번호        469361 non-null  object
 2   노선명         469361 non-null  object
 3   표준버스정류장ID   469361 non-null  int64 
 4   버스정류장ARS번호  469361 non-null  object
 5   역명          469361 non-null  object
 6   00시승차총승객수   469361 non-null  int64 
 7   00시하차총승객수   469361 non-null  int64 
 8   1시승차총승객수    469361 non-null  int64 
 9   1시하차총승객수    469361 non-null  int64 
 10  2시승차총승객수    469361 non-null  int64 
 11  2시하차총승객수    469361 non-null  int64 
 12  3시승차총승객수    469361 non-null  int64 
 13  3시하차총승객수    469361 non-null  int64 
 14  4시승차총승객수    469361 non-null  int64 
 15  4시하차총승객수    469361 non-null  int64 
 16  5시승차총승객수    469361 non-null  int64 
 17  5시하차총승객수    469361 non-null  int64 
 18  6시승차총승객수    469361 non-null  int64 
 19  6시하차총승객수    469361 non-

In [6]:
boarding.tail()

,사용년월,노선번호,노선명,표준버스정류장ID,버스정류장ARS번호,역명,00시승차총승객수,00시하차총승객수,1시승차총승객수,1시하차총승객수,...,19시하차총승객수,20시승차총승객수,20시하차총승객수,21시승차총승객수,21시하차총승객수,22시승차총승객수,22시하차총승객수,23시승차총승객수,23시하차총승객수,등록일자
469356,202112,6648,6648(방화동~양천구청),999800002,~,김포교통(종점가상),0,0,0,0,...,0,1,0,0,2,0,0,0,0,20220103
469357,202112,9404,9404번(분당구미~신사역),999800003,~,구미동차고지(기점가상),0,2,0,2,...,2,2,3,0,3,2,5,1,2,20220103
469358,202112,9404,9404번(분당구미~신사역),999800004,~,구미동차고지(종점가상),0,14,0,4,...,2,0,3,0,6,0,15,0,4,20220103
469359,202112,9707,9707번(고양 가좌동~영등포역),999800005,~,가좌동종점(종점가상),0,5,0,4,...,6,1,4,0,2,0,0,0,1,20220103
469360,202112,9701,9701번(가좌동~서울역),999800005,~,가좌동종점(종점가상),0,5,0,3,...,3,0,4,0,2,0,2,0,2,20220103


In [8]:
drop_cols = ['노선번호', '노선명', '역명', '등록일자', '00시하차총승객수', '1시하차총승객수', '2시하차총승객수', '3시하차총승객수', '4시하차총승객수', '5시하차총승객수'
             , '6시하차총승객수', '7시하차총승객수', '8시하차총승객수', '9시하차총승객수', '10시하차총승객수', '11시하차총승객수', '12시하차총승객수', '13시하차총승객수'
             , '14시하차총승객수', '15시하차총승객수', '16시하차총승객수', '17시하차총승객수', '18시하차총승객수', '19시하차총승객수', '20시하차총승객수', '21시하차총승객수'
             , '22시하차총승객수', '23시하차총승객수']
boarding = boarding.drop(drop_cols, axis=1)

boarding.head()

,사용년월,표준버스정류장ID,버스정류장ARS번호,00시승차총승객수,1시승차총승객수,2시승차총승객수,3시승차총승객수,4시승차총승객수,5시승차총승객수,6시승차총승객수,...,14시승차총승객수,15시승차총승객수,16시승차총승객수,17시승차총승객수,18시승차총승객수,19시승차총승객수,20시승차총승객수,21시승차총승객수,22시승차총승객수,23시승차총승객수
0,202101,100000002,01002,0,0,0,0,85,153,22,...,89,110,92,101,90,60,38,25,30,18
1,202101,100000003,01003,0,0,0,0,61,98,49,...,122,114,98,112,95,70,79,68,43,11
2,202101,100000005,01005,0,0,0,0,64,76,35,...,48,46,54,41,43,25,21,20,17,1
3,202101,100000102,01198,0,0,0,0,25,60,29,...,103,107,133,209,394,167,113,95,53,14
4,202101,100000108,01204,8,0,0,0,0,0,3,...,31,49,51,117,193,70,50,70,34,11


In [76]:
merge = pd.merge(boarding, station_gwangjin, how='inner', on='표준버스정류장ID')

In [77]:
merge.to_csv('./data/gwangjin_boarding.csv', index=False, encoding='cp949')

In [78]:
data = pd.read_csv('./data/gwangjin_boarding.csv', encoding='cp949')

In [79]:
data.head()

,사용년월,표준버스정류장ID,버스정류장ARS번호,00시승차총승객수,1시승차총승객수,2시승차총승객수,3시승차총승객수,4시승차총승객수,5시승차총승객수,6시승차총승객수,...,19시승차총승객수,20시승차총승객수,21시승차총승객수,22시승차총승객수,23시승차총승객수,위도,경도,버스도착정보안내기_설치_여부,구이름,행정동
0,202101,104000001,5001,84,0,0,0,0,36,206,...,472,522,659,293,199,127.075646,37.558741,설치,광진구,중곡1동
1,202101,104000001,5001,39,59,42,34,133,0,0,...,0,0,0,0,9,127.075646,37.558741,설치,광진구,중곡1동
2,202101,104000001,5001,0,0,0,0,27,81,87,...,260,119,156,75,15,127.075646,37.558741,설치,광진구,중곡1동
3,202101,104000001,5001,0,0,0,0,0,75,72,...,257,178,160,55,14,127.075646,37.558741,설치,광진구,중곡1동
4,202101,104000001,5001,0,0,0,0,409,503,205,...,232,167,256,136,70,127.075646,37.558741,설치,광진구,중곡1동


In [80]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9102 entries, 0 to 9101
Data columns (total 32 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   사용년월             9102 non-null   int64  
 1   표준버스정류장ID        9102 non-null   int64  
 2   버스정류장ARS번호       9102 non-null   int64  
 3   00시승차총승객수        9102 non-null   int64  
 4   1시승차총승객수         9102 non-null   int64  
 5   2시승차총승객수         9102 non-null   int64  
 6   3시승차총승객수         9102 non-null   int64  
 7   4시승차총승객수         9102 non-null   int64  
 8   5시승차총승객수         9102 non-null   int64  
 9   6시승차총승객수         9102 non-null   int64  
 10  7시승차총승객수         9102 non-null   int64  
 11  8시승차총승객수         9102 non-null   int64  
 12  9시승차총승객수         9102 non-null   int64  
 13  10시승차총승객수        9102 non-null   int64  
 14  11시승차총승객수        9102 non-null   int64  
 15  12시승차총승객수        9102 non-null   int64  
 16  13시승차총승객수        9102 non-null   int64  
 17  14시승차총승객수     

In [81]:
data.columns

Index(['사용년월', '표준버스정류장ID', '버스정류장ARS번호', '00시승차총승객수', '1시승차총승객수', '2시승차총승객수',
       '3시승차총승객수', '4시승차총승객수', '5시승차총승객수', '6시승차총승객수', '7시승차총승객수', '8시승차총승객수',
       '9시승차총승객수', '10시승차총승객수', '11시승차총승객수', '12시승차총승객수', '13시승차총승객수',
       '14시승차총승객수', '15시승차총승객수', '16시승차총승객수', '17시승차총승객수', '18시승차총승객수',
       '19시승차총승객수', '20시승차총승객수', '21시승차총승객수', '22시승차총승객수', '23시승차총승객수', '위도',
       '경도', '버스도착정보안내기_설치_여부', '구이름', '행정동'],
      dtype='object')

In [82]:
data.describe()

,사용년월,표준버스정류장ID,버스정류장ARS번호,00시승차총승객수,1시승차총승객수,2시승차총승객수,3시승차총승객수,4시승차총승객수,5시승차총승객수,6시승차총승객수,...,16시승차총승객수,17시승차총승객수,18시승차총승객수,19시승차총승객수,20시승차총승객수,21시승차총승객수,22시승차총승객수,23시승차총승객수,위도,경도
count,9102.000000,9.102000e+03,9102.000000,9102.000000,9102.000000,9102.000000,9102.000000,9102.000000,9102.000000,9102.000000,...,9102.000000,9102.000000,9102.000000,9102.000000,9102.000000,9102.000000,9102.000000,9102.000000,9102.000000,9102.000000
mean,202106.540101,1.041258e+08,5241.911009,16.358383,4.674028,2.629971,2.347726,23.243902,54.651615,103.271918,...,207.022962,221.997913,260.725445,199.963854,149.573610,150.306526,134.181608,51.911448,127.080899,37.548478
std,3.456431,3.971807e+05,188.594090,55.754602,23.890241,13.683858,16.501522,68.137587,103.514365,132.631096,...,260.550066,301.362870,421.943464,355.143371,266.015257,277.119609,263.656977,104.709649,0.009432,0.010885
min,202101.000000,1.010003e+08,5001.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,127.061162,37.530369
25%,202104.000000,1.040000e+08,5135.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,...,35.000000,36.000000,37.000000,27.000000,18.000000,16.000000,12.000000,2.000000,127.073628,37.538919
50%,202107.000000,1.040001e+08,5202.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,57.000000,...,136.000000,144.000000,151.000000,110.000000,76.000000,70.000000,58.000000,19.000000,127.081413,37.547585
75%,202110.000000,1.040002e+08,5248.000000,10.000000,0.000000,0.000000,0.000000,5.000000,73.000000,148.000000,...,275.000000,287.000000,317.000000,227.000000,170.000000,167.000000,143.000000,54.000000,127.086968,37.557156
max,202112.000000,1.049001e+08,5999.000000,1728.000000,702.000000,265.000000,531.000000,925.000000,1037.000000,1088.000000,...,2535.000000,3429.000000,5368.000000,4796.000000,3520.000000,4134.000000,3722.000000,1962.000000,127.110462,37.571462


In [2]:
data = pd.read_csv('./data/gwangjin_bus_station.csv')

In [4]:
data.head(2)

,정류장_ID,위도,경도,버스도착정보안내기_설치_여부,구이름,행정동
0,277103552,127.067091,37.531727,미설치,광진구,자양3동
1,277103551,127.066757,37.531816,미설치,광진구,자양4동


In [6]:
data = pd.read_csv('./data/gwangjin_bus_station.csv', encoding='utf-8')